In [28]:
team_abb = {
"ATLANTA HAWKS" : "ATL",
"ST. LOUIS HAWKS" : "SLH",
"MILWAUKEE HAWKS" : "MIL",
"TRI-CITIES BLACKHAWKS" : "TCB",
"BOSTON CELTICS" : "BOS",
"BROOKLYN NETS" : "BRK",
"NEW JERSEY NETS" : "NJN",
"CHICAGO BULLS" : "CHI",
"CHARLOTTE HORNETS": "CHH",
"CHARLOTTE BOBCATS" : "CHA",
"CLEVELAND CAVALIERS" : "CLE",
"DALLAS MAVERICKS" : "DAL",
"DENVER NUGGETS" : "DEN",
"DETROIT PISTONS" : "DET",
"FORT WAYNE PISTONS" : "FWP",
"GOLDEN STATE WARRIORS" : "GSW",
"SAN FRANCISCO WARRIORS" : "SFW",
"PHILADELPHIA WARRIORS" : "PHI",
"HOUSTON ROCKETS" : "HOU",
"INDIANA PACERS" : "IND",
"LOS ANGELES CLIPPERS" : "LAC",
"SAN DIEGO CLIPPERS" : "SDC",
"BUFFALO BRAVES" : "BUF",
"LOS ANGELES LAKERS" : "LAL",
"MINNEAPOLIS LAKERS" : "MIN",
"MEMPHIS GRIZZLIES" : "MEM",
"VANCOUVER GRIZZLIES" : "VAN",
"MIAMI HEAT" : "MIA",
"MILWAUKEE BUCKS" : "MIL",
"MINNESOTA TIMBERWOLVES" : "MIN",
"NEW ORLEANS PELICANS" : "NOP",
"NEW ORLEANS/OKLAHOMA CITY HORNETS" : "NOK",
"NEW ORLEANS HORNETS" : "NOH",
"NEW YORK KNICKS" : "NYK",
"OKLAHOMA CITY THUNDER" : "OKC",
"SEATTLE SUPERSONICS" : "SEA",
"ORLANDO MAGIC" : "ORL",
"PHILADELPHIA 76ERS" : "PHI",
"SYRACUSE NATIONALS" : "SYR",
"PHOENIX SUNS" : "PHO",
"PORTLAND TRAIL BLAZERS" : "POR",
"SACRAMENTO KINGS" : "SAC",
"KANSAS CITY KINGS" : "KCK",
"KANSAS CITY-OMAHA KINGS" : "KCK",
"CINCINNATI ROYALS" : "CIN",
"ROCHESTER ROYALS" : "ROR",
"SAN ANTONIO SPURS" : "SAS",
"TORONTO RAPTORS" : "TOR",
"UTAH JAZZ" : "UTA",
"NEW ORLEANS JAZZ" : "NOJ",
"WASHINGTON WIZARDS" : "WAS",
"WASHINGTON BULLETS" : "WAS",
"CAPITAL BULLETS" : "CAP",
"BALTIMORE BULLETS" : "BAL",
"CHICAGO ZEPHYRS" : "CHI",
"CHICAGO PACKERS" : "CHI",
"ANDERSON PACKERS" : "AND",
"CHICAGO STAGS" : "CHI",
"INDIANAPOLIS OLYMPIANS" : "IND",
"SHEBOYGAN RED SKINS" : "SRS",
"ST. LOUIS BOMBERS" : "SLB",
"WASHINGTON CAPITOLS" : "WAS",
"WATERLOO HAWKS" : "WAT",
"SAN DIEGO ROCKETS" : "SDR"
}

In [ ]:
import pandas as pd
# Look at the stats from last year to predict this year

season_df = pd.read_csv("seasons_00_19.csv")
players_df = pd.read_csv("players_00_19.csv")
teams_df = pd.read_csv("teamStats.csv")

all_df = pd.DataFrame()
for i, g in season_df.iterrows():
  date = str(g["DATE"])[:10]
  year = int(date[:4])
  if year < 2010:
    continue
  teamAway = team_abb[g["VISITOR"].upper()]
  teamHome = team_abb[g["HOME"].upper()]
  gameId = (date + teamHome + teamAway).replace("-","")
  game_df = pd.DataFrame(columns=["GAMEID", "HOME", "AWAY","HOMEWIN"])

  r = [gameId, teamHome, teamAway]
  r.append(True if g["HOME_PTS"] > g["VISITOR_PTS"] else False)
  game_df.loc[0]=r
  homeLastStats = teams_df.loc[(teams_df["TEAM"] == teamHome) & (teams_df["SEASON"] == year-1)]
  homeLastStats = homeLastStats.loc[:,"FG":"PTS"].add_prefix("HOME_").reset_index()
  awayLastStats = teams_df.loc[(teams_df["TEAM"] == teamAway) & (teams_df["SEASON"] == year-1)]
  awayLastStats = awayLastStats.loc[:,"FG":"PTS"].add_prefix("AWAY_").reset_index()
  game_df = pd.concat([game_df, homeLastStats, awayLastStats],axis=1).drop("index",axis=1)
  all_df = pd.concat([all_df, game_df])
all_df = all_df.dropna(axis=0)
all_df["HOMEWIN"] = all_df["HOMEWIN"].astype(int)
all_df.index = range(len(all_df))

In [122]:
all_df

,GAMEID,HOME,AWAY,HOMEWIN,HOME_FG,HOME_FGA,HOME_FG%,HOME_3P,HOME_3PA,HOME_3P%,...,AWAY_FT%,AWAY_ORB,AWAY_DRB,AWAY_TRB,AWAY_AST,AWAY_STL,AWAY_BLK,AWAY_TOV,AWAY_PF,AWAY_PTS
0,20100101ATLNYK,0,20,0,36.0,78.7,0.458,7.3,19.9,0.366,...,0.784,11.1,31.0,42.1,21.2,7.4,2.5,14.3,20.4,105.2
1,20100101MINORL,17,22,0,36.4,82.5,0.441,6.6,18.8,0.353,...,0.715,10.0,33.3,43.3,19.4,7.0,5.4,13.9,20.3,101.0
2,20100101LALSAC,13,26,1,40.3,85.1,0.474,6.7,18.5,0.361,...,0.798,10.2,28.9,39.1,19.7,6.9,4.3,15.4,23.3,100.6
3,20100102NJNCLE,18,5,0,35.7,79.7,0.448,8.0,21.2,0.376,...,0.757,10.8,31.4,42.2,20.3,7.2,5.3,12.7,20.3,100.3
4,20100102MIACHA,15,3,0,37.0,81.0,0.457,7.1,19.9,0.357,...,0.740,10.8,28.9,39.7,21.2,7.1,4.8,15.6,21.4,93.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10665,20190410PHICHI,23,4,1,40.8,86.6,0.472,11.0,29.8,0.369,...,0.759,9.6,35.0,44.7,23.5,7.6,3.5,14.0,19.2,102.9
10666,20190410SASDAL,27,6,1,39.0,85.4,0.457,8.5,24.1,0.352,...,0.763,8.1,33.1,41.3,22.7,7.0,3.8,12.3,19.2,102.3
10667,20190410DENMIN,7,17,1,40.7,86.6,0.470,11.5,30.9,0.371,...,0.804,10.3,31.6,42.0,22.7,8.4,4.2,12.5,18.2,109.5
10668,20190410LACUTA,12,29,1,40.3,85.4,0.471,9.5,26.8,0.354,...,0.779,9.0,34.2,43.3,22.4,8.6,5.1,14.7,19.6,104.1


In [31]:
import numpy as np
unique_teams = np.unique(all_df["HOME"].values)

In [121]:
for i, row in all_df.iterrows():
  all_df.at[i,"HOME"] = np.where(unique_teams == row["HOME"])[0][0]
  all_df.at[i,"AWAY"] = np.where(unique_teams == row["AWAY"])[0][0]

In [105]:
#one - hot
#classification_only_home = pd.DataFrame(0, index=np.arange(len(all_df)), columns = unique_teams)
#classification_only_away = pd.DataFrame(0, index=np.arange(len(all_df)), columns = unique_teams)
#j = 0
#for i, row in all_df.iterrows():
#  classification_only_home.at[j, row[1]] = 1
#  classification_only_away.at[j, row[2]] = 1
#  j+=1
#
#classification_only_home = classification_only_home.add_prefix("HOME_")
#classification_only_away = classification_only_away.add_prefix("AWAY_")



In [ ]:
!pip install elosports

In [123]:
all_df.insert(4,"HOME_ELO",1500)
all_df.insert(5,"AWAY_ELO",1500)

In [124]:
from elosports.elo import Elo
eloLeague = Elo(k = 20)
for t in range(0,31):
  eloLeague.addPlayer(t,rating=1500)

test_df = all_df.copy()
for i, row in test_df.iterrows():
  test_df.at[i,"HOME_ELO"] = eloLeague.ratingDict[row["HOME"]]
  test_df.at[i,"AWAY_ELO"] = eloLeague.ratingDict[row["AWAY"]]
  if row["HOMEWIN"] == 1:
    eloLeague.gameOver(row["HOME"], row["AWAY"],True)
  else:
    eloLeague.gameOver(row["AWAY"], row["HOME"],False)

In [16]:
#Export data
'''
eloLeague = Elo(k = 20)
for t in unique_teams:
  eloLeague.addPlayer(t,rating=1500)

elo_df = pd.DataFrame(columns=["Date","Team","Elo"])
test_df = all_df.copy()
for i, row in test_df.iterrows():
  r = []
  date = row["GAMEID"][:8]
  date = pd.to_datetime(date,yearfirst=True,format="%Y%m%d")
  r.append(date)
  r.append(row["HOME"])
  r.append(eloLeague.ratingDict[row["HOME"]])
  elo_df.loc[len(elo_df)]=r
  r = []
  r.append(date)
  r.append(row["AWAY"])
  r.append(eloLeague.ratingDict[row["AWAY"]])
  elo_df.loc[len(elo_df)]=r
  test_df.at[i,"HOME_ELO"] = eloLeague.ratingDict[row["HOME"]]
  test_df.at[i,"AWAY_ELO"] = eloLeague.ratingDict[row["AWAY"]]
  if row["HOMEWIN"] == 1:
    eloLeague.gameOver(row["HOME"], row["AWAY"],True)
  else:
    eloLeague.gameOver(row["AWAY"], row["HOME"],False)
    '''

In [27]:
#elo_df2 = elo_df.sort_values(by=['Team', 'Date'])
#elo_df2.to_csv("eloOverTime.csv")

In [126]:
#test_df = test_df[["HOME","AWAY","HOME_ELO","AWAY_ELO","HOMEWIN"]]
test_df = test_df.drop(["GAMEID"],axis=1)

In [139]:
test_df

,HOME,AWAY,HOMEWIN,HOME_ELO,AWAY_ELO,HOME_FG,HOME_FGA,HOME_FG%,HOME_3P,HOME_3PA,...,AWAY_FT%,AWAY_ORB,AWAY_DRB,AWAY_TRB,AWAY_AST,AWAY_STL,AWAY_BLK,AWAY_TOV,AWAY_PF,AWAY_PTS
0,0,20,0,1500,1500,36.0,78.7,0.458,7.3,19.9,...,0.784,11.1,31.0,42.1,21.2,7.4,2.5,14.3,20.4,105.2
1,17,22,0,1500,1500,36.4,82.5,0.441,6.6,18.8,...,0.715,10.0,33.3,43.3,19.4,7.0,5.4,13.9,20.3,101.0
2,13,26,1,1500,1500,40.3,85.1,0.474,6.7,18.5,...,0.798,10.2,28.9,39.1,19.7,6.9,4.3,15.4,23.3,100.6
3,18,5,0,1500,1500,35.7,79.7,0.448,8.0,21.2,...,0.757,10.8,31.4,42.2,20.3,7.2,5.3,12.7,20.3,100.3
4,15,3,0,1500,1500,37.0,81.0,0.457,7.1,19.9,...,0.740,10.8,28.9,39.7,21.2,7.1,4.8,15.6,21.4,93.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10665,23,4,1,1570,1319,40.8,86.6,0.472,11.0,29.8,...,0.759,9.6,35.0,44.7,23.5,7.6,3.5,14.0,19.2,102.9
10666,27,6,1,1605,1394,39.0,85.4,0.457,8.5,24.1,...,0.763,8.1,33.1,41.3,22.7,7.0,3.8,12.3,19.2,102.3
10667,7,17,1,1622,1466,40.7,86.6,0.470,11.5,30.9,...,0.804,10.3,31.6,42.0,22.7,8.4,4.2,12.5,18.2,109.5
10668,12,29,1,1554,1621,40.3,85.4,0.471,9.5,26.8,...,0.779,9.0,34.2,43.3,22.4,8.6,5.1,14.7,19.6,104.1


In [ ]:
#export_df = test_df[["GAMEID","HOME_ELO","AWAY_ELO"]].copy()
#export_df.to_csv("elo.csv")

In [ ]:
#temp1 = classification_only_home.join(classification_only_away)
#temp2 = all_df.drop(["GAMEID", "HOME","AWAY"],axis=1)
#temp2 = test_df[["HOMEWIN","HOME_ELO","AWAY_ELO"]].copy()
#new_df = pd.concat([temp1,temp2],axis=1)
#new_df

In [59]:
import pandas as pd
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [149]:
classification = test_df.copy()

In [150]:
y = classification['HOMEWIN']
X = classification.drop(['HOMEWIN'], axis = 1)
X = normalize(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=13)

In [144]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=20)

knn = knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))
#confusion_matrix(y_test,y_pred)

Accuracy:  0.6433926897844423
Precision: 0.675812274368231
Recall: 0.75


In [188]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(30,30,2), activation='tanh', solver='adam', max_iter=300)
mlp.fit(X_train,y_train)

predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)
print(accuracy_score(y_test,predict_test))

0.6541705716963448
